# Basic EM workflow 3 (Restaurants data set)

# Introduction

This IPython notebook explains a basic workflow two tables using *py_entitymatching*. Our goal is to come up with a workflow to match restaurants from Fodors and Zagat sites. Specifically, we want to maximize F1. The datasets contain information about the restaurants.

First, we need to import *py_entitymatching* package and other libraries as follows:

In [1]:
import sys
#sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]
pandas version: 0.22.0
magellan version: 0.3.0


Matching two tables typically consists of the following three steps:

** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# Read input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [3]:
# Get the paths
path_A = '/home/liang/Workspace/cs839-project/stage-2/crawlers/data/amazon_products.csv'
path_B = '/home/liang/Workspace/cs839-project/stage-2/crawlers/data/walmart_products.csv'

In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [5]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 2976
Number of tuples in B: 4847
Number of tuples in A X B (i.e the cartesian product): 14424672


In [6]:
A.head(2)

,id,product title,brand,model,combo
0,a0,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...
1,a1,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...,acer,e5-576g-5762,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...


In [7]:
B.head(2)

,id,product title,brand,model,combo
0,w0,iview i896qw 8.95 2-in-1 32gb tablet intel atom bay trail z3735f processor windows 10,iview,NaN,iview i896qw 8.95 2-in-1 32gb tablet intel atom bay trail z3735f processor windows 10 iview
1,w1,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red,dell,i3168-3270red,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...


In [8]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

In [9]:
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 200, 1, show_progress=True)
len(A1), len(B1)

# But for the purposes of this notebook, we will use the entire table A and B

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


(146, 200)

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two restaurants with different city names will not match. So we decide the apply blocking over names:

In [10]:
# Blocking plan

# A, B -- attribute equiv. blocker [city] --------------------|---> candidate set

# block on: brand, ram, OS?, model? 

In [11]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C1 = ob.block_tables(A, B, 'product title', 'product title', 
                    l_output_attrs=['product title'], 
                    r_output_attrs=['product title'],
                    overlap_size=75,
                    q_val=5,
                    word_level=False,
                    show_progress=True,
                    n_jobs=-1,
                    )
len(C1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


6513

In [12]:
C1

,_id,ltable_id,rtable_id,ltable_product title,rtable_product title
0,0,a2000,w6,hp stream with an ultra-portable design laptop 14 screen intel celeron n3060 processor 4gb ram 3...,11.6 convertible touchscreen laptop windows 10 home office 365 personal 1-year subscription incl...
1,1,a322,w32,lenovo flex 5 2-in-1 laptop: core i5-8250u 8gb ram 128gb ssd 14-inch full hd touch display windo...,lenovo flex 5 2-in-1 laptop: core i5-8250u 8gb ram 128gb ssd 14-inch full hd touch display windo...
2,2,a138,w41,asus vivobook flip 14 thin and light 2-in-1 hd touchscreen laptop intel 2.2ghz processor 4gb ram...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
3,3,a174,w41,asus vivobook e403na-us04 thin and lightweight 14” fhd laptop intel celeron n3350 processor 4gb ...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
4,4,a255,w41,asus vivobook flip tp200sa-dh01t 11.6 inch display thin and lightweight 2-in-1 hd touchscreen la...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
5,5,a621,w41,asus vivobook flip 14 tp401ca-dhm6t 14” thin and lightweight 2-in-1 hd touchscreen laptop intel ...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
6,6,a1167,w41,asus vivobook e203na ultra-thin and light laptop intel celeron n3350 processor 4gb lpddr3 ram 32...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
7,7,a648,w41,asus transformer book tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 full hd tou...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
8,8,a432,w72,dell inspiron 13 2-in-1 laptop: core i7-8550u 256gb ssd 8gb ram 13.3 full hd touch display windo...,dell inspiron 13 2-in-1 laptop: core i7-8550u 256gb ssd 8gb ram 13.3 full hd touch display
9,9,a1161,w76,lenovo yoga y720 13 premium thin light convertible 2 in 1 laptop pc (intel 8th gen i7 quad core ...,hp spectre x360 13t premium ultra light convertible 2-in-1 laptop/tablet (intel 8th gen quad cor...


## Debug blocker output

The number of tuple pairs considered for matching is reduced to 10165 (from 176423), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in *py_entitymatching* as follows:

In [13]:
# Debug blocker output
# dbg = em.debug_blocker(C1, A, B, output_size=200, 
#                        attr_corres=[
#                            ('product title','product title'),
#                            ('brand', 'brand'), 
#                            ('model','model'),
#                            ('combo','combo')],
#                       verbose=True)
#### Display first few tuple pairs from the debug_blocker's output
# dbg

From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the restaurants to match the  names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [15]:
# Updated blocking sequence
# A, B ------ attribute equivalence [city] -----> C1--
#                                                     |----> C
# A, B ------ overlap blocker [name] --------> C2--

In [16]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C2 = ob.block_tables(A, B, 'combo', 'combo', 
                    l_output_attrs=['product title', 'combo'], 
                    r_output_attrs=['product title', 'combo'],
                    overlap_size=125,
                    q_val=5,
                    word_level=False,
                    show_progress=True,
                    n_jobs=-1
                    )
len(C2)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


727

In [17]:
# Display first two rows from C2
C2

,_id,ltable_id,rtable_id,ltable_product title,ltable_combo,rtable_product title,rtable_combo
0,0,a255,w41,asus vivobook flip tp200sa-dh01t 11.6 inch display thin and lightweight 2-in-1 hd touchscreen la...,asus vivobook flip tp200sa-dh01t 11.6 inch display thin and lightweight 2-in-1 hd touchscreen la...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
1,1,a648,w41,asus transformer book tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 full hd tou...,asus transformer book tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 full hd tou...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
2,2,a1116,w99,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...
3,3,a1791,w99,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...
4,4,a1594,w108,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...
5,5,a2490,w108,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...
6,6,a1594,w110,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...
7,7,a2490,w110,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...
8,8,a116,w113,2018 newest hp envy x360 convertible 2-in-1 full hd ips 15.6 touchscreen notebook intel quad cor...,2018 newest hp envy x360 convertible 2-in-1 full hd ips 15.6 touchscreen notebook intel quad cor...,hp envy x360 convertible 2-in-1 full hd ips 15.6 touchscreen notebook intel core i7-8550u proces...,hp envy x360 convertible 2-in-1 full hd ips 15.6 touchscreen notebook intel core i7-8550u proces...
9,9,a1116,w114,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (a

We add another blocker.

In [18]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C3 = ob.block_tables(A, B, 'product title', 'product title', 
                    l_output_attrs=['product title'], 
                    r_output_attrs=['product title'],
                    overlap_size=20,
                    show_progress=True,
                    n_jobs=-1,
                    )
len(C3)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


2066

In [19]:
C3

,_id,ltable_id,rtable_id,ltable_product title,rtable_product title
0,0,a629,w36,dell i5368-1692gry 13.3 fhd 2-in-1 laptop (intel core i3-6100u 2.3ghz processor 4 gb ram 1 tb hd...,refurbished dell i5368-1692gry 13.3 fhd 2-in-1 laptop (intel core i3-6100u 2.3ghz processor 4 gb...
1,1,a255,w41,asus vivobook flip tp200sa-dh01t 11.6 inch display thin and lightweight 2-in-1 hd touchscreen la...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
2,2,a648,w41,asus transformer book tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 full hd tou...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
3,3,a555,w86,2018 latest hp spectre x360 13t touchscreen yoga style 2-in-1 windows 10 pro laptop & tablet - i...,hp spectre x360 13t premium ultra light convertible 2-in-1 laptop (intel 8th gen i7-8550u quad c...
4,4,a1116,w99,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...
5,5,a1437,w99,cuk hp envy x360 15z convertible touch notebook (amd ryzen 5 2500u + amd radeon vega 8 16gb ram ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...
6,6,a1791,w99,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...
7,7,a116,w107,2018 newest hp envy x360 convertible 2-in-1 full hd ips 15.6 touchscreen notebook intel quad cor...,hp envy x360 convertible 2-in-1 full hd ips 15.6 touchscreen notebook intel core i7-8550u proces...
8,8,a1594,w108,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...
9,9,a2490,w108,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...


In [20]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C4 = ob.block_tables(A, B, 'combo', 'combo', 
                    l_output_attrs=['product title', 'combo'], 
                    r_output_attrs=['product title', 'combo'],
                    overlap_size=20,
                    show_progress=True,
                    n_jobs=-1
                    )
len(C4)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


2158

In [21]:
C4

,_id,ltable_id,rtable_id,ltable_product title,ltable_combo,rtable_product title,rtable_combo
0,0,a629,w36,dell i5368-1692gry 13.3 fhd 2-in-1 laptop (intel core i3-6100u 2.3ghz processor 4 gb ram 1 tb hd...,dell i5368-1692gry 13.3 fhd 2-in-1 laptop (intel core i3-6100u 2.3ghz processor 4 gb ram 1 tb hd...,refurbished dell i5368-1692gry 13.3 fhd 2-in-1 laptop (intel core i3-6100u 2.3ghz processor 4 gb...,refurbished dell i5368-1692gry 13.3 fhd 2-in-1 laptop (intel core i3-6100u 2.3ghz processor 4 gb...
1,1,a255,w41,asus vivobook flip tp200sa-dh01t 11.6 inch display thin and lightweight 2-in-1 hd touchscreen la...,asus vivobook flip tp200sa-dh01t 11.6 inch display thin and lightweight 2-in-1 hd touchscreen la...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
2,2,a648,w41,asus transformer book tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 full hd tou...,asus transformer book tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 full hd tou...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
3,3,a555,w86,2018 latest hp spectre x360 13t touchscreen yoga style 2-in-1 windows 10 pro laptop & tablet - i...,2018 latest hp spectre x360 13t touchscreen yoga style 2-in-1 windows 10 pro laptop & tablet - i...,hp spectre x360 13t premium ultra light convertible 2-in-1 laptop (intel 8th gen i7-8550u quad c...,hp spectre x360 13t premium ultra light convertible 2-in-1 laptop (intel 8th gen i7-8550u quad c...
4,4,a1116,w99,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...
5,5,a1437,w99,cuk hp envy x360 15z convertible touch notebook (amd ryzen 5 2500u + amd radeon vega 8 16gb ram ...,cuk hp envy x360 15z convertible touch notebook (amd ryzen 5 2500u + amd radeon vega 8 16gb ram ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...
6,6,a1791,w99,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...
7,7,a116,w107,2018 newest hp envy x360 convertible 2-in-1 full hd ips 15.6 touchscreen notebook intel quad cor...,2018 newest hp envy x360 convertible 2-in-1 full hd ips 15.6 touchscreen notebook intel quad cor...,hp envy x360 convertible 2-in-1 full hd ips 15.6 touchscreen notebook intel core i7-8550u proces...,hp envy x360 convertible 2-in-1 full hd ips 15.6 touchscreen notebook intel core i7-8550u proces...
8,8,a1594,w108,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...
9,9,a2490,w108,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core fx 9800p apu amd radeon r7 ...,hp envy x360 15t 2-in-1 convertible laptop pc (amd 7th gen quad-core 

In [22]:
# Combine blocker outputs
C = em.combine_blocker_outputs_via_union([C1, C2, C3, C4])

In [23]:
len(C)

6713

We observe that the number of tuple pairs considered for matching is increased to 12530 (from 10165). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [24]:
# Debug again
dbg = em.debug_blocker(C, A, B, output_size=200)

In [25]:
# Display first few rows from the debugger output
dbg

,_id,ltable_id,rtable_id,ltable_product title,ltable_brand,ltable_model,ltable_combo,rtable_product title,rtable_brand,rtable_model,rtable_combo
0,0,a826,w2550,acer swift 3 sf314-52-517z 14 laptop computer - silver intel core i5-8250u processor 1.6ghz 8gb ...,acer,nx.gqgaa.002,acer swift 3 sf314-52-517z 14 laptop computer - silver intel core i5-8250u processor 1.6ghz 8gb ...,acer sf314-51-52w2 swift 3 14 laptop windows 10 intel core i5-6200u processor 8gb ram 256gb soli...,acer,nx.gkbaa.002,acer sf314-51-52w2 swift 3 14 laptop windows 10 intel core i5-6200u processor 8gb ram 256gb soli...
1,1,a229,w4803,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,dell,i3168-3271blu,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,refurbished dell i3552-3240blk 15.6 hd laptop (intel pentium n3700 1.6ghz processor 4 gb ddr3l s...,dell,i3552-3240blk,refurbished dell i3552-3240blk 15.6 hd laptop (intel pentium n3700 1.6ghz processor 4 gb ddr3l s...
2,2,a229,w2131,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,dell,i3168-3271blu,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,new dell i3552-3240blk 15.6 hd laptop (intel pentium n3700 1.6ghz processor 4 gb ddr3l sdram 500...,dell,i3552-3240blk,new dell i3552-3240blk 15.6 hd laptop (intel pentium n3700 1.6ghz processor 4 gb ddr3l sdram 500...
3,3,a1463,w2343,acer predator helios 15.6 laptop intel core i7 2.80ghz 16gb ram 1tb hdd 256gb ssd (certified ref...,acer,g3-572-72yf,acer predator helios 15.6 laptop intel core i7 2.80ghz 16gb ram 1tb hdd 256gb ssd (certified ref...,refurbished acer predator 17 laptop intel i7 2.80ghz 16gb ram 1tb hdd 256gb ssd win10home,acer,nh.q29aa.002,refurbished acer predator 17 laptop intel i7 2.80ghz 16gb ram 1tb hdd 256gb ssd win10home acer n...
4,4,a2110,w743,lg gram intel i7 16gb ram 15.6 touchscreen laptop dark silver (15z970-a.aas7u1) with software + ...,lg,NaN,lg gram intel i7 16gb ram 15.6 touchscreen laptop dark silver (15z970-a.aas7u1) with software + ...,lg 15z970-a.aas7u1 gram intel i7 16gb ram 15.6 touchscreen laptop dark silver,lg,15z970-a.aas7u1,lg 15z970-a.aas7u1 gram intel i7 16gb ram 15.6 touchscreen laptop dark silver lg 15z970-a.aas7u1
5,5,a1202,w4097,apple macbook air md760ll/a 13.3-inch laptop (old version),apple,md760ll/a,apple macbook air md760ll/a 13.3-inch laptop (old version) apple md760ll/a,apple macbook air md231ll/a 13.3-inch laptop (old version) (scratched and dented),apple,NaN,apple macbook air md231ll/a 13.3-inch laptop (old version) (scratched and dented) apple
6,6,a136,w2310,dell inspiron 15 gaming laptop: core i7-7700hq 16gb ram 128gb ssd and 1tb hdd gtx 1060 6gb 15.6-...,dell,i7577-7722blk,dell inspiron 15 gaming laptop: core i7-7700hq 16gb ram 128gb ssd and 1tb hdd gtx 1060 6gb 15.6-...,dell inspiron 15 gaming laptop: core i5-7300hq gtx 1060 6gb 128gb ssd+1tb hdd 8gb ram 15.6 full ...,dell,i7577-5258blk-pus,dell inspiron 15 gaming laptop: core i5-7300hq gtx 1060 6gb 128gb ssd+1tb hdd 8gb ram 15.6 full ...
7,7,a854,w585,hp pavilion 15-inch laptop intel core i5-7200u 8gb ram 1tb hard drive windows 10 (15-cc010nr sil...,hp,15-cc010nr,hp pavilion 15-inch laptop intel core i5-7200u 8gb ram 1tb hard drive windows 10 (15-cc010nr sil...,hp 15-bs030nr 15.6 laptop touchscreen windows 10 intel core i5-7200u processor 8gb ram 1tb hard ...,hp,1kv01ua#aba,hp 15-bs030nr 15.6 laptop touchscreen windows 10 intel core i5-7200u processor 8gb ram 1tb hard ...
8,8,a951,w2265,dell 4m5j9 latitude 3480 14 hd laptop (intel core i5-7200u 8gb ddr4 128gb solid state drive wind...,dell,4m5j9,dell 4m5j9 latitude 3480 14 hd laptop (intel core i5-7200u 8gb ddr4 128gb solid state drive wind...,dell latitude 3580 15.6 hd laptop intel core i5-7200u 8gb ddr4 128gb solid state drive windows 1...,dell,lati35802t78m,dell latitude 3580 15.6 hd laptop intel core i5-72

We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.


# Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following five steps:
1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [26]:
# Sample  candidate set
S = em.sample_table(C, 10)
S

,_id,ltable_id,rtable_id,ltable_product title,ltable_combo,rtable_product title,rtable_combo
972,972,a1347,w3127,lenovo thinkpad t470s windows 10 pro laptop - intel core i7-7600u 20gb ram 512gb ssd 14 ips fhd ...,lenovo thinkpad t470s windows 10 pro laptop - intel core i7-7600u 20gb ram 512gb ssd 14 ips fhd ...,pcie lenovo thinkpad t460s business windows 10 pro laptop - intel core i7-6600u 8gb ram 256gb pc...,pcie lenovo thinkpad t460s business windows 10 pro laptop - intel core i7-6600u 8gb ram 256gb pc...
1621,1621,a1513,w3312,lenovo thinkpad x1 yoga 20jf000dus 14 touchscreen lcd 2560 x 1440 - ips - intel core i7 (7th gen...,lenovo thinkpad x1 yoga 20jf000dus 14 touchscreen lcd 2560 x 1440 - ips - intel core i7 (7th gen...,refurbished lenovo thinkpad x1 yoga 20jf000hus 14 touchscreen lcd 2 in 1 ultrabook - intel core ...,refurbished lenovo thinkpad x1 yoga 20jf000hus 14 touchscreen lcd 2 in 1 ultrabook - intel core ...
1749,1749,a1602,w1283,hp - 17z laptop 17.3 with hd+ touchscreen ( amd dual-core a6-9220 apu amd radeon r4 graphics 16g...,hp - 17z laptop 17.3 with hd+ touchscreen ( amd dual-core a6-9220 apu amd radeon r4 graphics 16g...,hp - 17z laptop 17.3 with hd + touchscreen (amd dual-core a6-9220 apu amd radeon r4 graphics 8gb...,hp - 17z laptop 17.3 with hd + touchscreen (amd dual-core a6-9220 apu amd radeon r4 graphics 8gb...
1862,1862,a1716,w3182,lenovo thinkpad p51 mobile workstation laptop - windows 10 pro - intel quad-core i7-7820hq 64gb ...,lenovo thinkpad p51 mobile workstation laptop - windows 10 pro - intel quad-core i7-7820hq 64gb ...,lenovo thinkpad t460s business performance windows 10 pro laptop - intel core i7-6600u 20gb ram ...,lenovo thinkpad t460s business performance windows 10 pro laptop - intel core i7-6600u 20gb ram ...
4543,4543,a2418,w3380,lenovo thinkpad p50 mobile workstation laptop - windows 7 pro - intel i7-6700hq 64gb ram 2tb ssd...,lenovo thinkpad p50 mobile workstation laptop - windows 7 pro - intel i7-6700hq 64gb ram 2tb ssd...,lenovo thinkpad t460s business performance windows 8.1 pro laptop - intel core i7-6600u 20gb ram...,lenovo thinkpad t460s business performance windows 8.1 pro laptop - intel core i7-6600u 20gb ram...
4942,4942,a2439,w3377,lenovo thinkpad t470s windows 10 pro laptop - intel core i7-7600u 12gb ram 512gb ssd 14 ips fhd ...,lenovo thinkpad t470s windows 10 pro laptop - intel core i7-7600u 12gb ram 512gb ssd 14 ips fhd ...,pcie lenovo thinkpad t460s business windows 10 pro laptop - intel core i7-6600u 12gb ram 1tb pci...,pcie lenovo thinkpad t460s business windows 10 pro laptop - intel core i7-6600u 12gb ram 1tb pci...
5057,5057,a2534,w3025,lenovo thinkpad p51 15.6 4k uhd mobile workstation laptop pc (intel i7 quad core 32gb ram 1tb hd...,lenovo thinkpad p51 15.6 4k uhd mobile workstation laptop pc (intel i7 quad core 32gb ram 1tb hd...,lenovo thinkpad p51s 15.6'' premium mobile workstation ultrabook laptop (intel i7 processor 32gb...,lenovo thinkpad p51s 15.6'' premium mobile workstation ultrabook laptop (intel i7 processor 32gb...
5159,5159,a2636,w1630,hp envy 15t high performance laptop pc with full hd touchscreen ( intel i7 processor 16gb ram 1t...,hp envy 15t high performance laptop pc with full hd touchscreen ( intel i7 processor 16gb ram 1t...,hp pavilion 15t gaming laptop with full hd touchscreen ( intel i7 quad core 32gb nvidia geforce ...,hp pavilion 15t gaming laptop with full hd touchscreen ( intel i7 quad core 32gb nvidia geforce ...
6352,6352,a566,w714,2018 newest premium hp 15.6 business flagship laptop hd+ wled-backlit touchscreen display intel ...,2018 newest premium hp 15.6 business flagship laptop hd+ wled-backlit touchscreen display intel ...,2018 newest premium hp 15.6? touchscreen hd laptop intel dual core i3-7100u processor 2.40ghz 8g...,2018 newest premium hp 15.6? touchscreen hd laptop intel dual core i3-7100u processor 2.40ghz 8g...
6588,6588,a888,w3336,lenovo thinkpad x1 carbon 4 business ultrabook - windows 10 pro - intel core i7-6600u 1tb ss

Next, we label the sampled candidate set. Specify we would enter 1 for a match and 0 for a non-match.

In [27]:
# Label S
G = em.label_table(S, 'match')

Column name (match) is not present in dataframe
/usr/local/lib/python3.6/dist-packages/py_entitymatching/gui/table_gui.py:94: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  table.set_value(idxv[i], cols[j], val)


For the purposes of this guide, we will load in a pre-labeled dataset (of 450 tuple pairs) included in this package.

In [28]:
# # Load the pre-labeled data
# path_G = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/lbl_restnt_wf1.csv'
# G = em.read_csv_metadata(path_G, 
#                          key='_id',
#                          ltable=A, rtable=B, 
#                          fk_ltable='ltable_id', fk_rtable='rtable_id')
# len(G)
G

,_id,ltable_id,rtable_id,ltable_product title,ltable_combo,rtable_product title,rtable_combo,match
972,972,a1347,w3127,lenovo thinkpad t470s windows 10 pro laptop - intel core i7-7600u 20gb ram 512gb ssd 14 ips fhd ...,lenovo thinkpad t470s windows 10 pro laptop - intel core i7-7600u 20gb ram 512gb ssd 14 ips fhd ...,pcie lenovo thinkpad t460s business windows 10 pro laptop - intel core i7-6600u 8gb ram 256gb pc...,pcie lenovo thinkpad t460s business windows 10 pro laptop - intel core i7-6600u 8gb ram 256gb pc...,0
1621,1621,a1513,w3312,lenovo thinkpad x1 yoga 20jf000dus 14 touchscreen lcd 2560 x 1440 - ips - intel core i7 (7th gen...,lenovo thinkpad x1 yoga 20jf000dus 14 touchscreen lcd 2560 x 1440 - ips - intel core i7 (7th gen...,refurbished lenovo thinkpad x1 yoga 20jf000hus 14 touchscreen lcd 2 in 1 ultrabook - intel core ...,refurbished lenovo thinkpad x1 yoga 20jf000hus 14 touchscreen lcd 2 in 1 ultrabook - intel core ...,0
1749,1749,a1602,w1283,hp - 17z laptop 17.3 with hd+ touchscreen ( amd dual-core a6-9220 apu amd radeon r4 graphics 16g...,hp - 17z laptop 17.3 with hd+ touchscreen ( amd dual-core a6-9220 apu amd radeon r4 graphics 16g...,hp - 17z laptop 17.3 with hd + touchscreen (amd dual-core a6-9220 apu amd radeon r4 graphics 8gb...,hp - 17z laptop 17.3 with hd + touchscreen (amd dual-core a6-9220 apu amd radeon r4 graphics 8gb...,0
1862,1862,a1716,w3182,lenovo thinkpad p51 mobile workstation laptop - windows 10 pro - intel quad-core i7-7820hq 64gb ...,lenovo thinkpad p51 mobile workstation laptop - windows 10 pro - intel quad-core i7-7820hq 64gb ...,lenovo thinkpad t460s business performance windows 10 pro laptop - intel core i7-6600u 20gb ram ...,lenovo thinkpad t460s business performance windows 10 pro laptop - intel core i7-6600u 20gb ram ...,0
4543,4543,a2418,w3380,lenovo thinkpad p50 mobile workstation laptop - windows 7 pro - intel i7-6700hq 64gb ram 2tb ssd...,lenovo thinkpad p50 mobile workstation laptop - windows 7 pro - intel i7-6700hq 64gb ram 2tb ssd...,lenovo thinkpad t460s business performance windows 8.1 pro laptop - intel core i7-6600u 20gb ram...,lenovo thinkpad t460s business performance windows 8.1 pro laptop - intel core i7-6600u 20gb ram...,0
4942,4942,a2439,w3377,lenovo thinkpad t470s windows 10 pro laptop - intel core i7-7600u 12gb ram 512gb ssd 14 ips fhd ...,lenovo thinkpad t470s windows 10 pro laptop - intel core i7-7600u 12gb ram 512gb ssd 14 ips fhd ...,pcie lenovo thinkpad t460s business windows 10 pro laptop - intel core i7-6600u 12gb ram 1tb pci...,pcie lenovo thinkpad t460s business windows 10 pro laptop - intel core i7-6600u 12gb ram 1tb pci...,0
5057,5057,a2534,w3025,lenovo thinkpad p51 15.6 4k uhd mobile workstation laptop pc (intel i7 quad core 32gb ram 1tb hd...,lenovo thinkpad p51 15.6 4k uhd mobile workstation laptop pc (intel i7 quad core 32gb ram 1tb hd...,lenovo thinkpad p51s 15.6'' premium mobile workstation ultrabook laptop (intel i7 processor 32gb...,lenovo thinkpad p51s 15.6'' premium mobile workstation ultrabook laptop (intel i7 processor 32gb...,1
5159,5159,a2636,w1630,hp envy 15t high performance laptop pc with full hd touchscreen ( intel i7 processor 16gb ram 1t...,hp envy 15t high performance laptop pc with full hd touchscreen ( intel i7 processor 16gb ram 1t...,hp pavilion 15t gaming laptop with full hd touchscreen ( intel i7 quad core 32gb nvidia geforce ...,hp pavilion 15t gaming laptop with full hd touchscreen ( intel i7 quad core 32gb nvidia geforce ...,1
6352,6352,a566,w714,2018 newest premium hp 15.6 business flagship laptop hd+ wled-backlit touchscreen display intel ...,2018 newest premium hp 15.6 business flagship laptop hd+ wled-backlit touchscreen display intel ...,2018 newest premium hp 15.6? touchscreen hd laptop intel dual core i3-7100u processor 2.40ghz 8g...,2018 newest premium hp 15.6? touchscreen hd laptop intel dual core i3-7100u processor 2.40ghz 8g...,1
6588,6588,a888,w3336,lenovo thinkpad x1 carbon 4 business ultrabook - windows 10 pro - in

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [29]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers

In [30]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [41]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=True)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,id,id,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
1,product title,product title,short string (1 word),short string (1 word),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,brand,brand,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
3,model,model,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,combo,combo,short string (1 word),short string (1 word),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"


Do you want to proceed? (y/n):n

If the attribute correspondences or types have been inferred incorrectly,
use the get_features() function with your  own correspondences and attribute
types to get the correct features for your data


In [32]:
# List the names of the features generated
feature_table['feature_name']

0                                      id_id_lev_dist
1                                       id_id_lev_sim
2                                           id_id_jar
3                                           id_id_jwn
4                                           id_id_exm
5                               id_id_jac_qgm_3_qgm_3
6         product_title_product_title_jac_qgm_3_qgm_3
7     product_title_product_title_cos_dlm_dc0_dlm_dc0
8                         brand_brand_jac_qgm_3_qgm_3
9                     brand_brand_cos_dlm_dc0_dlm_dc0
10                    brand_brand_jac_dlm_dc0_dlm_dc0
11                                    brand_brand_mel
12                               brand_brand_lev_dist
13                                brand_brand_lev_sim
14                                    brand_brand_nmw
15                                     brand_brand_sw
16                        model_model_jac_qgm_3_qgm_3
17                    model_model_cos_dlm_dc0_dlm_dc0
18                    model_

### Converting the development set to feature vectors

In [34]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='match',
                            show_progress=False)

In [35]:
# Display first few rows
H.head(3)

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,product_title_product_title_jac_qgm_3_qgm_3,...,model_model_cos_dlm_dc0_dlm_dc0,model_model_jac_dlm_dc0_dlm_dc0,model_model_mel,model_model_lev_dist,model_model_lev_sim,model_model_nmw,model_model_sw,combo_combo_jac_qgm_3_qgm_3,combo_combo_cos_dlm_dc0_dlm_dc0,match
6588,6588,a888,w3336,5,0.0,0.000000,0.000000,0,0.0,0.888298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.890052,0.949289,1
1621,1621,a1513,w3312,4,0.2,0.466667,0.466667,0,0.0,0.615000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.567442,0.743484,0
5057,5057,a2534,w3025,5,0.0,0.000000,0.000000,0,0.0,0.497872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.490909,0.612903,1


### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' and 'recall' metric to select the best matcher.

In [37]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/dist-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.6/runpy.py in _run_code(code=<

### Debugging matcher

We observe that the best matcher is not maximizing F1. We debug the matcher to see what might be wrong.
To do this, first we split the feature vectors into train and test.

In [38]:
#  Split feature vectors into train and test
UV = em.split_train_test(H, train_proportion=0.5)
U = UV['train']
V = UV['test']

Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher for debugging purposes.

In [40]:
# Debug decision tree using GUI
em.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        target_attr='match')

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

From the GUI, we observe that phone numbers seem to be an important attribute, but they are in different format. Current features does not capture and adding a feature incorporating this difference in format can potentially improve 
the F1 numbers.

In [ ]:
def phone_phone_feature(ltuple, rtuple):
    p1 = ltuple.phone
    p2 = rtuple.phone
    p1 = p1.replace('/','-')
    p1 = p1.replace(' ','')
    p2 = p2.replace('/','-')
    p2 = p2.replace(' ','')    
    if p1 == p2:
        return 1.0
    else:
        return 0.0

In [ ]:
feature_table = em.get_features_for_matching(A, B)
em.add_blackbox_feature(feature_table, 'phone_phone_feature', phone_phone_feature)

Now, we repeat extracting feature vectors (this time with updated feature table), imputing table and selecting the best matcher again using cross-validation.

In [ ]:
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_after='gold', show_progress=False)

In [ ]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

Now, observe the best matcher is achieving a better F1. Let us stop here and  proceed on to evaluating the best matcher on the unseen data (the evaluation set).

##  Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [ ]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='gold', show_progress=False)

### Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [ ]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [ ]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [ ]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold', 'predicted')
em.print_eval_summary(eval_result)